## Step -1 Business Problem Understanding

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('Advertising.csv')
df.head()

,TV,radio,newspaper,sales
0,230100,37800,69200,22100
1,44500,39300,45100,10400
2,17200,45900,69300,9300
3,151500,41300,58500,18500
4,180800,10800,58400,12900


## everything is same as we did in multiple linear regression

In [4]:
X = df.drop(columns='sales')
y = df['sales']

## Step - 4: Modeling

### findout the best random_state value 

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score

Train = []
Test = []
CV = []

for i in range(0, 100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)
    model = LinearRegression()
    model.fit(X_train, y_train)

    ypred_train = model.predict(X_train)
    ypred_test = model.predict(X_test)

    Train.append(model.score(X_train, y_train))
    # print('Train R2:', r2_score(y_train, ypred_train))
    Test.append(model.score(X_test, y_test))
    # print('Test R2:', r2_score(y_test, ypred_test))

    CV.append(cross_val_score(model, X_train, y_train, cv=5).mean())


em = pd.DataFrame({'Train':Train, 'Test':Test, 'CV':CV})
gm = em[(abs(em['Train']-em['Test']) <= 0.05) & (abs(em['Test']-em['CV']) <=0.05)]
print('best random state number:', gm[gm['Test']==gm['Test'].max()].index.to_list())

best random state number: [99]


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

# <font color = aqua> ElasticNet Regresion </font> 

#### ElasticNet Regression with default parameters

In [7]:
from sklearn.linear_model import ElasticNet
enr_base = ElasticNet()
enr_base.fit(X_train, y_train)

print('Intercept:', enr_base.intercept_)
print('coefficient:', enr_base.coef_)

# Predictions
ypred_train = enr_base.predict(X_train)
ypred_test = enr_base.predict(X_test)

# Evaluation
from sklearn.metrics import r2_score
print('Train R2:', r2_score(y_train, ypred_train))
print('Test R2:', r2_score(y_test, ypred_test))
from sklearn.model_selection import cross_val_score
print('CV Score:', cross_val_score(enr_base, X_train, y_train, cv=5).mean())

Intercept: 2904.4859918962647
coefficient: [0.04550574 0.18820546 0.00075244]
Train R2: 0.8906288862925658
Test R2: 0.9200846682238386
CV Score: 0.8745389865770152


#### Apply Hyperparameter tuning for ElastiNet Regression

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

estimator = ElasticNet()
# Parameters & values
param_grid = {'alpha':[0.1, 0.2, 1, 2, 3, 5, 10], 'l1_ratio':[0.1, 0.5, 0.75, 0.9, 0.95, 1]}
# Identifying the best value of the parameter within given values for the given data
model_hp = GridSearchCV(estimator, param_grid, cv=5, scoring='neg_mean_squared_error')
model_hp.fit(X_train, y_train)
model_hp.best_params_ 

{'alpha': 10, 'l1_ratio': 1}

#### Rebuilt ElasticNet Model using best hyperparameters

In [9]:
from sklearn.linear_model import ElasticNet
enr_best = ElasticNet(alpha=10, l1_ratio=1)
enr_best.fit(X_train, y_train)

print('Intercept:', enr_best.intercept_)
print('Coefficients:', enr_best.coef_)

# Prediction 
ypred_train = enr_best.predict(X_train)
ypred_test = enr_best.predict(X_test)

# Evaluation
from sklearn.metrics import r2_score
print('Train R2:', r2_score(y_train, ypred_train))
print('Test R2:', r2_score(y_test, ypred_test))
from sklearn.model_selection import cross_val_score
print('CV Score:', cross_val_score(enr_best, X_train, y_train, cv=5).mean())

Intercept: 2904.4873169633047
Coefficients: [0.04550574 0.18820543 0.00075243]
Train R2: 0.8906288862925474
Test R2: 0.9200846724542091
CV Score: 0.8745390126429001


In [10]:
import statsmodels.formula.api as smf
model1 = smf.ols('y_train ~ X_train', data=X_train).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                y_train   R-squared:                       0.891
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                     423.4
Date:                Fri, 02 Feb 2024   Prob (F-statistic):           1.02e-74
Time:                        10:17:37   Log-Likelihood:                -1419.4
No. Observations:                 160   AIC:                             2847.
Df Residuals:                     156   BIC:                             2859.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2904.4859    368.721      7.877      0.000    2176.156    3632.816
X_train[0]     0.0455      0.002     28.499      0.000       0.042       0.049
X_train[1]     0.1882      0.010     18.941      0.000       0.169       0.208
X_train[2]     0.0008      0.007      0.114      0.909      -0.012       0.014
==============================================================================
Omnibus:                       53.148   Durbin-Watson:                   1.945
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              134.731
Skew:                          -1.386   Prob(JB):                     5.54e-30
Kurtosis:                       6.539   Cond. No.                     4.75e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.75e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Final model including TV and radio only 

In [11]:
X = X.drop(columns=['newspaper'])

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

In [13]:
from sklearn.linear_model import ElasticNet
enr_best = ElasticNet(alpha=10, l1_ratio=1)
enr_best.fit(X_train, y_train)

print('Intercept:', enr_best.intercept_)
print('Coefficients:', enr_best.coef_)

# Prediction 
ypred_train = enr_best.predict(X_train)
ypred_test = enr_best.predict(X_test)

# Evaluation
from sklearn.metrics import r2_score
print('Train R2:', r2_score(y_train, ypred_train))
print('Test R2:', r2_score(y_test, ypred_test))
from sklearn.model_selection import cross_val_score
print('CV Score:', cross_val_score(enr_best, X_train, y_train, cv=5).mean())

Intercept: 2918.21692154003
Coefficients: [0.04550633 0.1886176 ]
Train R2: 0.8906197253189078
Test R2: 0.9202930353903783
CV Score: 0.8803158278328016


### Predictions on a new data

In [14]:
test_df = pd.DataFrame({'TV':[149000], 'radio':[22000], 'newspaper':[12000]})
test_df

,TV,radio,newspaper
0,149000,22000,12000


In [15]:
X = test_df.drop(columns=['newspaper'])

In [16]:
enr_best.predict(X)

array([13848.24714622])

### Multiple Linear regression
- Test R2: 0.9314
- sales = 13886.112 (sales with 93.14% accuarcy)

### Polynomial regression
- Test R2: 0.9860
- sales = 14541.816 (sales with 98.60% accuarcy)

### Lasso regression
- Test R2: 0.8515
- sales = 13859.373 (sales with 85.15% accuarcy)

### Ridge regression
- Test R2: 0.8515
- sales = 13859.372 (sales with 85.15% accuarcy)

### ElasticNet regression
- Test R2: 0.8354
- sales = 13964.746 (sales with 83.54% accuarcy)